In [ ]:
#!pip install google.generativeai pip install 
#!pip install langchain-google-genai


In [1]:
from dotenv import load_dotenv
import os
local=True
load_dotenv(".env_generic")
load_dotenv(".env_neo4j_local")
model_file=".env_gemini"  
load_dotenv(model_file)
print(os.getenv('OPENAI_MODEL_NAME'),os.getenv('OPENAI_API_BASE'))

gemini-1.5-flash https://api.openai.com/v1


In [2]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import GraphCypherQAChain
from langchain.agents import Tool
from langchain_community.graphs import Neo4jGraph

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
#model="gemini-pro"
model="gemini-1.5-flash"
llm = ChatGoogleGenerativeAI(model=model, google_api_key=GOOGLE_API_KEY,temperature=0, verbose=True,top_k=200)
graph = Neo4jGraph(url=os.environ['NEO4J_URI'], username=os.environ['NEO4J_USERNAME'], password=os.environ['NEO4J_PASSWORD'])
graph.refresh_schema()

CYPHER_GENERATION_TEMPLATE = """
Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
All the data is based on ifc (industry foundation classes) standard.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Cypher examples:
# get gross volume of all entities that has gross volume (such as IfcBeam, IfcWallStandardCase, IfcSlab, IfcColumn): 
             MATCH (n:IfcQuantityVolume)<-[:Quantities]-(:IfcElementQuantity)<-[:RelatingPropertyDefinition]-(:IfcRelDefinesByProperties)-[:RelatedObjects]->(o) return labels(o)[0] as entity_name,o.ObjectType as entity_type ,n.VolumeValue as gross_volume
# get all columns with relation to floors:
            MATCH (n:IfcBuildingStorey)<-[:RelatingStructure]-(ss:IfcRelContainedInSpatialStructure)-[:RelatedElements]->(c:IfcColumn) RETURN n,ss,c
# whar are the materials used for beam?
             match (mt:IfcMaterial)<-[:RelatingMaterial]-(ascmat:IfcRelAssociatesMaterial)-[:RelatedObjects]->(bm:IfcBeam ) return (mt) .
# what objects were made by materials ?
            MATCH (m:IfcMaterial)<-[:RelatingMaterial]-(asm:IfcRelAssociatesMaterial)-[:RelatedObjects]->(o) return (o)
# what spaces spaces are in floors?
            match(IfcBuildingStorey)<-[:RelatingObject]-(:IfcRelAggregates)-[:RelatedObjects]->(s:IfcSpace) return (s)
# count beams by type:
            MATCH (b:IfcBeam) RETURN b.ObjectType AS object_type, COUNT(b) AS count ORDER BY count DESC
# count columns by type:
            MATCH (b:IfcColumns) RETURN b.ObjectType AS object_type, COUNT(b) AS count ORDER BY count DESC
# count walls by type:
            MATCH (b:IfcWall) RETURN b.ObjectType AS object_type, COUNT(b) AS count ORDER BY count DESC
# get the buildings floor/storey sequence, Storey Level, Storey Elevation and Storey Name:
            MATCH (storey:IfcBuildingStorey) WHERE storey.Elevation IS NOT NULL WITH storey ORDER BY storey.Elevation ASC WITH collect(storey) AS sortedStoreys UNWIND range(0, size(sortedStoreys) - 1) AS idx WITH idx, sortedStoreys[idx] AS storey, sortedStoreys RETURN CASE WHEN storey.Elevation < 0 THEN -1 * (size([s IN sortedStoreys WHERE s.Elevation < 0]) - idx) WHEN storey.Elevation = 0 THEN 0 ELSE idx - size([s IN sortedStoreys WHERE s.Elevation < 0]) END AS StoreyLevel, idx+1 as StoreySequence,storey.Name AS StoreyName, storey.Elevation AS Elevation ORDER BY storey.Elevation ASC

# what are the relation between IfcBuilding and  IfcBuildingStorey storeys:
            MATCH (b:IfcBuilding)<-[:RelatingObject]-(:IfcRelAggregates)-[:RelatedObjects]->(bs:IfcBuildingStorey) return(bs)
# get the building top floor/storey name:
            MATCH (storey:IfcBuildingStorey) WHERE storey.Elevation IS NOT NULL WITH storey ORDER BY storey.Elevation DESC LIMIT 1 RETURN storey.Name AS TopStoreyName
# get the  storey level of the middle storey:
            MATCH (storey:IfcBuildingStorey) WHERE storey.Elevation IS NOT NULL WITH storey ORDER BY storey.Elevation ASC WITH collect(storey) AS sortedStoreys WITH sortedStoreys, size(sortedStoreys) AS totalCount WITH sortedStoreys, totalCount, floor(totalCount / 2.0) AS middleIndex RETURN sortedStoreys[toInteger(middleIndex)] AS middleStorey, sortedStoreys[toInteger(middleIndex)].Name AS StoreyName, sortedStoreys[toInteger(middleIndex)].Elevation AS Elevation
# get the number of columns, sum of column volume, sum of beam volume,  average of beam volume,  average of column volume for each storey of the building, mention the storey name and elevation order by elevation:
            MATCH (storey:IfcBuildingStorey)<-[:RelatingStructure]-(ss:IfcRelContainedInSpatialStructure)-[:RelatedElements]->(c:IfcColumn) OPTIONAL MATCH (c)<-[:RelatedObjects]-(eq:IfcRelDefinesByProperties)-[:RelatingPropertyDefinition]->(ev:IfcElementQuantity)-[:Quantities]->(qv:IfcQuantityVolume) WITH storey, count(c) AS column_count, sum(qv.VolumeValue) AS column_volume_sum, storey.Name AS storey_name, storey.Elevation AS storey_elevation OPTIONAL MATCH (storey)<-[:RelatingStructure]-()-[:RelatedElements]->(b:IfcBeam) OPTIONAL MATCH (b)<-[:RelatedObjects]-()-[:RelatingPropertyDefinition]->(ev:IfcElementQuantity)-[:Quantities]->(qv:IfcQuantityVolume) WITH storey, column_count, column_volume_sum, storey_name, storey_elevation, count(b) AS beam_count, sum(qv.VolumeValue) AS beam_volume_sum RETURN storey_name, storey_elevation, column_count, column_volume_sum, beam_count, beam_volume_sum, CASE WHEN beam_count > 0 THEN beam_volume_sum / beam_count ELSE 0 END AS beam_volume_avg, CASE WHEN column_count > 0 THEN column_volume_sum / column_count ELSE 0 END AS column_volume_avg ORDER BY storey_elevation ASC

Critical Note: 
Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
{question}"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

neo4jquery = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT
)
neo4j_query_tool = Tool(
  name="neo4j_query_tool",
  func=neo4jquery.run,
  description="neo4j database that contains data about the project, user provide english question the tool translates it to query and respond with the answer",
  cypher_prompt=CYPHER_GENERATION_PROMPT
)
#neo4j_query_tool.run("how many doors?")

I0000 00:00:1723204061.016930 3625558 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache, work_serializer_dispatch
I0000 00:00:1723204061.026687 3625558 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723204061.028197 3625558 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [3]:
#neo4j_query_tool.run("get the number of columns, sum of column volume, sum of beam volume,  average of beam volume,  average of column volume for each storey of the building, mention the storey name and elevation order by elevation ")

In [ ]:
# Agent 1: Researcher
from crewai import Agent, Task, Crew
analyst = Agent(
    role="civil engineer",
    goal="provide information about building materials, quantities and sizes, be as accurate as possible",
    llm=llm,
    tools = [  neo4j_query_tool],
    verbose=True,
    backstory=(
        "As civil engineer, you get your input from the neo4j_query_tool, you ask the questions in human language, for example, if you want to know number of floors you ask : how many floors are there? and not a cypher that you think that is right "
        "as the tool translates the query better than you. "
        "The tool has arythmetic capabilities, and you should use it as much as possible. "
        "The tool has information that is pre calculated, so you dont need to perform the calculations for things, like volumes. "
        "usually the tool should respond to the questions, including aggregation and sums. only if you are not satisfied from the answer, you might need to calculate things by yourself, according to the data you get from the tool. you should not invent anything!!! "
        "sometimes you need to conclude things about entities from their name or ObjectType. and then you should get the value and conclude by yourself. "
        "If you would need to conclude something from column name by yourself you should do it, the default is not to do so. "
        "for example, if you are requested to count beams by type and for each type get the properties, such as: material, shape or dimensions, you shold follow the following steps: "
        "1) request from tool: count number of beams by ObjectType. "
        "2) using the information you got from the tool, look at the value of ObjectType, and conclude fron the text what are the properties "
        "3) only if you dont find the requested properties, try to get it be other relationships. "
        "getting data for walls and columns, should follow  similiar process as of the beam example."
    )
)
# Task for Researcher Agent:
analyst_task = Task(
    description=(
#        "get the columns and their types, for each type, calculate how many instances. the ObjectType property each wall type is a description of the column. identify the material,  Shape,  Dimensions"
        "provide number and volume of beams by floor"
    ),
    expected_output=(
#        "json with the information of types with their properties from the text of the type. count how many of each type. "
#        "do not write enything else except for this json"
        "json with the result by floor"
    ),
    agent=analyst,
    async_execution=True
)
application_crew = Crew(
    agents=[analyst],

    tasks=[analyst_task],

    verbose=True
)
result = application_crew.kickoff()

 [2024-08-09 14:47:48][DEBUG]: == Working Agent: civil engineer
 [2024-08-09 14:47:48][INFO]: == Starting Task: provide number and volume of beams by floor


> Entering new CrewAgentExecutor chain...


I0000 00:00:1723204069.144476 3625779 work_stealing_thread_pool.cc:321] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1723204069.144633 3625779 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


Thought: I need to find out the number and volume of beams per floor. I can do this by querying the database for the number of beams per floor and then getting the volume of each beam type. 

Action: neo4j_query_tool
Action Input: {"tool_input": "How many beams are there per floor?"}

> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (b:IfcBeam)<-[:RelatedElements]-(ss:IfcRelContainedInSpatialStructure)-[:RelatingStructure]->(f:IfcBuildingStorey) RETURN f.Name AS floor_name, COUNT(b) AS beam_count ORDER BY floor_name

Full Context:
[{'floor_name': 'גג טכני', 'beam_count': 50}, {'floor_name': 'גלריה 0', 'beam_count': 6}, {'floor_name': 'גלריה 1', 'beam_count': 28}, {'floor_name': 'טכני עליון', 'beam_count': 52}, {'floor_name': 'קומה 1', 'beam_count': 94}, {'floor_name': 'קומה 10', 'beam_count': 62}, {'floor_name': 'קומה 11', 'beam_count': 61}, {'floor_name': 'קומה 12', 'beam_count': 61}, {'floor_name': 'קומה 13', 'beam_count': 62}, {'floor_name': 'קומה 15', 'beam

2024-08-09 14:48:26,793 - 123145657012224 - before_sleep.py-before_sleep:65 - WARNING: Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


 

I encountered an error while trying to use the tool. This was the error: 429 Resource has been exhausted (e.g. check quota)..
 Tool neo4j_query_tool accepts these inputs: neo4j database that contains data about the project, user provide english question the tool translates it to query and respond with the answer



2024-08-09 14:48:29,322 - 123145657012224 - before_sleep.py-before_sleep:65 - WARNING: Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..




> Entering new CrewAgentExecutor chain...


2024-08-09 14:48:31,792 - 123145657012224 - before_sleep.py-before_sleep:65 - WARNING: Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..




> Entering new CrewAgentExecutor chain...


2024-08-09 14:48:34,350 - 123145657012224 - before_sleep.py-before_sleep:65 - WARNING: Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Exception in thread Thread-7 (_execute_task_async):
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/usr/local/anaconda3/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda3/lib/python3.11/site-packages/crewai/task.py", line 206, in _execute_task_async
    result = self._execute_core(agent, context, tools)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/anaconda3/lib/python3.11/site-packages/crewai/task.py", line 228, in _execute_core
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/anacond

In [ ]:
result.raw

In [ ]:
import google.generativeai as genai
import os
#analyzePrompt = """This neo4j schema contains a . Return output in json format:
#{description: description, features: [feature1, feature2, feature3]}"""

model = genai.GenerativeModel(model_name="gemini-1.5-flash")
#response = model.generate_content([analyzePrompt, img])
#print(response.text)

In [ ]:
model.generate_content("what is ifc?")

In [ ]:
from langchain import OpenAI, LLMChain
from langchain.prompts import PromptTemplate

# Replace with your actual Gemini API key

# Define a prompt template
prompt_template = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

# Create an LLMChain using Gemini
llm_chain = LLMChain(
    llm=model,
    prompt=prompt_template,
)

# Ask a question
question = "What is the capital of France?"
response = llm_chain.run(question)

print(response)

In [ ]:
!pip install langchain --upgrade

In [ ]:
from langchain import PromptTemplate, LLMChain
import requests
from google.api_core.client_options import ClientOptions

class GeminiLLM:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://generativeai.googleapis.com/v1beta"

    def _call_gemini(self, prompt):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }
        data = {
            "prompt": prompt,
            "model": "text-bison@001",  # Replace with your desired Gemini model
        }
        response = requests.post(f"{self.base_url}/models:generateText", headers=headers, json=data)
        response.raise_for_status()
        return response.json()["text"]

    def predict(self, prompt):
        return self._call_gemini(prompt)

# Example usage
prompt_template = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

gemini_llm = GeminiLLM(api_key=os.environ["GOOGLE_API_KEY"])
#llm_chain = LLMChain(prompt=prompt_template, llm=gemini_llm)

#question = "What is the capital of France?"
#response = llm_chain.run(question)
#print(response)


In [ ]:
model.generate_content("Give me python code to sort a list")